# Herdnet

In [1]:
from datalabeling.train.herdnet import HerdnetData, HerdnetTrainer
from datalabeling.arguments import Arguments
import lightning as L
from tqdm import tqdm

c:\Users\FADELCO\miniconda3\envs\label-backend\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
data = HerdnetData(data_config_yaml=r"..\data\data_config.yaml",
                   patch_size=640,
                   batch_size=4)
data.setup(stage="fit")

In [ ]:
for idx in tqdm(range(len(data.train_dataset))):
    _ = data.train_dataset[idx]

In [ ]:
img,(heatmap_target,clsmap_target) = data.train_dataset[0]

In [ ]:
img.shape, heatmap_target.shape, clsmap_target.shape

In [ ]:
for batch in data.train_dataloader():
    batch_img, (batch_heatmap_target,batch_clsmap_target) = batch

In [ ]:
img, target = data.val_dataset[0]

In [ ]:
target

In [ ]:
for batch in data.val_dataloader():
    pass

In [ ]:
val_img,val_target = batch

In [ ]:
val_img.shape

In [ ]:
val_target

In [2]:
args = Arguments()
args.data_config_yaml = r"..\data\data_config.yaml"
args.lr0 = 3e-4

In [3]:
herndet_trainer = HerdnetTrainer(herdnet_model_path=r"D:\datalabeling\models\20220329_HerdNet_Ennedi_dataset_2023.pth",
                      args=args,
                      ce_weight=None,
                      work_dir="../.tmp"
                    )

In [4]:
datamodule = HerdnetData(data_config_yaml=r"..\data\data_config.yaml",
                   patch_size=640,
                   batch_size=4)
datamodule.setup("fit")

Getting groundtruths: 592it [00:01, 415.63it/s]0:00<?, ?it/s]
concatenating datasets: 100%|██████████| 1/1 [00:01<00:00,  1.47s/it]
Getting groundtruths: 592it [00:01, 428.61it/s]0:00<?, ?it/s]
concatenating datasets: 100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


In [ ]:
# trainer = L.Trainer(limit_train_batches=100, 
#                     num_sanity_val_steps=0,
#                     max_epochs=1,
#                 )

# trainer.fit(model=herndet_trainer,datamodule=datamodule
#         )

In [ ]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '/content/output'
mkdir(work_dir)

lr = 1e-4
weight_decay = 1e-3
epochs = 100

optimizer = Adam(params=herndet_trainer.model.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=datamodule.num_classes)

stitcher = HerdNetStitcher(
    model=herdnet, 
    size=(patch_size,patch_size), 
    overlap=160, 
    down_ratio=down_ratio, 
    reduction='mean'
    )

evaluator = HerdNetEvaluator(
    model=herdnet, 
    dataloader=val_dataloader, 
    metrics=metrics, 
    stitcher=stitcher, 
    work_dir=work_dir, 
    header='validation'
    )

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=evaluator,
    work_dir=work_dir
    )

# PaddlePaddle